In [1]:
!pip install torch==2.5.1
!pip install transformers datasets accelerate bitsandbytes
!pip install unsloth
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
import pandas as pd
from datetime import datetime
from transformers import TrainingArguments, Trainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Verify GPU
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")

CUDA Available: True
GPU Device: Tesla T4


In [4]:
# Initialize model with unsloth and PEFT
from peft import LoraConfig, get_peft_model

# First initialize the model
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
MAX_LENGTH = 128

# Initialize base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_LENGTH,
    dtype=torch.bfloat16,
    load_in_4bit=True,
    trust_remote_code=True,
  #  attn_implementation="eager" ,
    use_cache=False,
    # device_map="auto"
)

# Disable xformers attention
model.config.use_cache = False
model.config.pretraining_tp = 1
model.config.use_flash_attention = False

# Add LoRA adapter configuration
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Add LoRA adapters to the model
model = get_peft_model(model, lora_config)
model.enable_input_require_grads()
model.gradient_checkpointing_enable()
model.print_trainable_parameters()  # This will show you the trainable parameters

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device does not support bfloat16. Will change to float16.


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


In [5]:
# Create training dataset, this is an example, You can load from hugging face
training_data = {
    'text': [
        "### Human: Translate to Hindi colloquial: What are you doing this weekend?\n### Assistant: aap is saptaahaant par kya kar rahe hain?",
        "### Human: Translate to Hindi colloquial: How's your current project going?\n### Assistant: aapaka vartamaan projekt kaisa chal raha hai?",
        "### Human: Translate to Hindi colloquial: How are you feeling today?\n### Assistant: aaj aap kaisa mahasoos kar rahe hain?",
        "### Human: Translate to Hindi colloquial: What's your favorite childhood memory?\n### Assistant: aapakee pasandeeda bachapan kee smrti kya hai?",
        "### Human: Translate to Hindi colloquial: What's your favorite movie and why?\n### Assistant: aapakee pasandeeda philm kaun see hai aur kyon?",
        "### Human: Translate to Hindi colloquial: what you are doing?\n### Assistant: aap kya kar rahe hain",
        "### Human: Translate to Hindi colloquial: how is your health?\n### Assistant: aapakee tabeeyat kaisee hai?",
        "### Human: Translate to Hindi colloquial: How are you doing?\n### Assistant: aap kaise hain?",
        "### Human: Translate to Hindi colloquial: Where are you going?\n### Assistant: aap kahaan ja rahe hain?",
        "### Human: Translate to Hindi colloquial: What time is it?\n### Assistant: ye vaqt kya hai?",
        "### Human: Translate to Hindi colloquial: The food there is never any good.\n### Assistant: vahaan ka khaana kabhee bhee achchha nahin hota.",
        "### Human: Translate to Hindi colloquial: I could not walk any farther.\n### Assistant: main aur aage nahin chal saka.",
        "### Human: Translate to Hindi colloquial: I can't eat any more pizza.\n### Assistant: main aur pizza nahin kha sakata.",
        "### Human: Translate to Hindi colloquial: What was the highlight of your day so far?\n### Assistant: aapake ab tak ke din ka mukhy aakarshan kya tha?",
        "### Human: Translate to Hindi colloquial: Can you explain this concept?\n### Assistant: kya aap is avadhaarana ko samajha sakate hain?",
        "### Human: Translate to Hindi colloquial: How does this work?\n### Assistant: kaise yah kaam karata hai?",
        "### Human: Translate to Hindi colloquial: What is Deep learning?\n### Assistant: deep larning kya hai?",
        "### Human: Translate to Hindi colloquial: Where can I find the documentation?\n### Assistant: mujhe dastaavej kahaan mil sakate hain?",
        "### Human: Translate to Hindi colloquial: Why is this not working?\n### Assistant: yah kaam kyon nahin kar raha hai?",
        "### Human: Translate to Hindi colloquial: Can you explain this concept?\n### Assistant: kya aap is avadhaarana ko samajha sakate hain?",
        "### Human: Translate to Hindi colloquial: How does this work?\n### Assistant: kaise yah kaam karata hai?",
        "### Human: Translate to Hindi colloquial: What is Deep learning?\n### Assistant: deep larning kya hai?",
        "### Human: Translate to Hindi colloquial: Where can I find the documentation?\n### Assistant: mujhe dastaavej kahaan mil sakate hain?",
        "### Human: Translate to Hindi colloquial: Why is this not working?\n### Assistant: yah kaam kyon nahin kar raha hai?"
    ]

}

# Create dataset
dataset = Dataset.from_dict(training_data)
dataset = dataset.shuffle(seed=42)
split_dataset = dataset.train_test_split(test_size=0.2)

# ADD THIS NEW PREPROCESSING FUNCTION HERE
def preprocess_function(examples):
    # Tokenize inputs
    model_inputs = tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
        # return_tensors="pt"
    )

    # Create labels (same as input_ids for causal language modeling)
    model_inputs['labels'] = model_inputs['input_ids']

    return model_inputs

# THEN ADD THIS PREPROCESSING STEP
tokenized_train = split_dataset['train'].map(
    preprocess_function,
    remove_columns=['text'],
    batched=True
)
tokenized_val = split_dataset['test'].map(
    preprocess_function,
    remove_columns=['text'],
    batched=True
)

# Convert to PyTorch format AFTER preprocessing
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


print(f"Training examples: {len(split_dataset['train'])}")
print(f"Validation examples: {len(split_dataset['test'])}")

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Training examples: 19
Validation examples: 5


In [6]:
# Create training dataset, this is an example, You can load from hugging face
training_data = {
    'text': [
        "### Human: Translate to Hindi colloquial: What are you doing this weekend?\n### Assistant: aap is saptaahaant par kya kar rahe hain?",
        "### Human: Translate to Hindi colloquial: How's your current project going?\n### Assistant: aapaka vartamaan projekt kaisa chal raha hai?",
        "### Human: Translate to Hindi colloquial: How are you feeling today?\n### Assistant: aaj aap kaisa mahasoos kar rahe hain?",
        "### Human: Translate to Hindi colloquial: What's your favorite childhood memory?\n### Assistant: aapakee pasandeeda bachapan kee smrti kya hai?",
        "### Human: Translate to Hindi colloquial: What's your favorite movie and why?\n### Assistant: aapakee pasandeeda philm kaun see hai aur kyon?",
        "### Human: Translate to Hindi colloquial: what you are doing?\n### Assistant: aap kya kar rahe hain",
        "### Human: Translate to Hindi colloquial: how is your health?\n### Assistant: aapakee tabeeyat kaisee hai?",
        "### Human: Translate to Hindi colloquial: How are you doing?\n### Assistant: aap kaise hain?",
        "### Human: Translate to Hindi colloquial: Where are you going?\n### Assistant: aap kahaan ja rahe hain?",
        "### Human: Translate to Hindi colloquial: What time is it?\n### Assistant: ye vaqt kya hai?",
        "### Human: Translate to Hindi colloquial: The food there is never any good.\n### Assistant: vahaan ka khaana kabhee bhee achchha nahin hota.",
        "### Human: Translate to Hindi colloquial: I could not walk any farther.\n### Assistant: main aur aage nahin chal saka.",
        "### Human: Translate to Hindi colloquial: I can't eat any more pizza.\n### Assistant: main aur pizza nahin kha sakata.",
        "### Human: Translate to Hindi colloquial: What was the highlight of your day so far?\n### Assistant: aapake ab tak ke din ka mukhy aakarshan kya tha?",
        "### Human: Translate to Hindi colloquial: Can you explain this concept?\n### Assistant: kya aap is avadhaarana ko samajha sakate hain?",
        "### Human: Translate to Hindi colloquial: How does this work?\n### Assistant: kaise yah kaam karata hai?",
        "### Human: Translate to Hindi colloquial: What is Deep learning?\n### Assistant: deep larning kya hai?",
        "### Human: Translate to Hindi colloquial: Where can I find the documentation?\n### Assistant: mujhe dastaavej kahaan mil sakate hain?",
        "### Human: Translate to Hindi colloquial: Why is this not working?\n### Assistant: yah kaam kyon nahin kar raha hai?",
        "### Human: Translate to Hindi colloquial: Can you explain this concept?\n### Assistant: kya aap is avadhaarana ko samajha sakate hain?",
        "### Human: Translate to Hindi colloquial: How does this work?\n### Assistant: kaise yah kaam karata hai?",
        "### Human: Translate to Hindi colloquial: What is Deep learning?\n### Assistant: deep larning kya hai?",
        "### Human: Translate to Hindi colloquial: Where can I find the documentation?\n### Assistant: mujhe dastaavej kahaan mil sakate hain?",
        "### Human: Translate to Hindi colloquial: Why is this not working?\n### Assistant: yah kaam kyon nahin kar raha hai?"
    ]

}

# Create dataset
dataset = Dataset.from_dict(training_data)
dataset = dataset.shuffle(seed=42)
split_dataset = dataset.train_test_split(test_size=0.2)

# ADD THIS NEW PREPROCESSING FUNCTION HERE
def preprocess_function(examples):
    # Tokenize inputs
    model_inputs = tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
        # return_tensors="pt"
    )

    # Create labels (same as input_ids for causal language modeling)
    model_inputs['labels'] = model_inputs['input_ids']

    return model_inputs

# THEN ADD THIS PREPROCESSING STEP
tokenized_train = split_dataset['train'].map(
    preprocess_function,
    remove_columns=['text'],
    batched=True
)
tokenized_val = split_dataset['test'].map(
    preprocess_function,
    remove_columns=['text'],
    batched=True
)

# Convert to PyTorch format AFTER preprocessing
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


print(f"Training examples: {len(split_dataset['train'])}")
print(f"Validation examples: {len(split_dataset['test'])}")

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Training examples: 19
Validation examples: 5


In [23]:
# Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val
)

In [25]:
print("Starting training...")
trainer.train()


Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 19 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 57
 "-____-"     Number of trainable parameters = 4,505,600


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sravanigaddam1405 (sravanigaddam1405-vignana-bharathi-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,14.549300
20,14.260000
30,13.594100
40,12.817500
50,11.824400


TrainOutput(global_step=57, training_loss=13.081969277900562, metrics={'train_runtime': 266.4611, 'train_samples_per_second': 0.214, 'train_steps_per_second': 0.214, 'total_flos': 45484051267584.0, 'train_loss': 13.081969277900562, 'epoch': 3.0})

In [26]:
trainer.save_model()
trainer.push_to_hub()


No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/AriaKage/TinyLlama-Hindi-Colloquial-Translator/commit/27071a28ca52b75b0371fc74335f5c7d14b1369b', commit_message='End of training', commit_description='', oid='27071a28ca52b75b0371fc74335f5c7d14b1369b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AriaKage/TinyLlama-Hindi-Colloquial-Translator', endpoint='https://huggingface.co', repo_type='model', repo_id='AriaKage/TinyLlama-Hindi-Colloquial-Translator'), pr_revision=None, pr_num=None)

In [27]:
print("Preparing model for inference...")
model = FastLanguageModel.for_inference(model)


Preparing model for inference...


In [28]:
def translate_to_Hindi_colloquial(text):
    prompt = f"""### Human: You are a Hindi colloquial language translator. Translate the following English text to Hindi colloquial language (spoken Tamil).
Here are some examples:
"What are you doing this weekend?" -> "aap is saptaahaant par kya kar rahe hain?"
"How's your current project going?" -> " aapaka vartamaan projekt kaisa chal raha hai?"
"How are you feeling today?" -> "aaj aap kaisa mahasoos kar rahe hain?"
"What's your favorite childhood memory?" -> "aapakee pasandeeda bachapan kee smrti kya hai?"
"What's your favorite movie and why?" -> "aapakee pasandeeda philm kaun see hai aur kyon?"
"Where are you going?" -> "aap kahaan ja rahe hain?"
"how is your health?" -> "aapakee tabeeyat kaisee hai?"
"How are you doing?" -> "aap kaise hain?"
"Where are you going?" -> "aap kahaan ja rahe hain?"
"What time is it?" -> "ye vaqt kya hai?"
"The food there is never any good." -> "vahaan ka khaana kabhee bhee achchha nahin hota."
"I could not walk any farther." -> " main aur aage nahin chal saka."
"I can't eat any more pizza." -> "main aur pizza nahin kha sakata."


Now translate this: {text}
### Assistant: """
    inputs = tokenizer(prompt, return_tensors="pt", max_length=MAX_LENGTH, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Increase max_length to allow for generated tokens in addition to the input length
    outputs = model.generate(
        **inputs,
        max_length=MAX_LENGTH + 64,  # Allow 64 additional tokens for generation
        do_sample=True,          # Enable sampling
        temperature=0.7,         # Keep temperature
        num_return_sequences=1,  # Return only one sequence
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        num_beams=1,            # Disable beam search
        top_p=0.95,             # Add top_p sampling
        repetition_penalty=1.2   # Add repetition penalty
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("### Assistant:")[-1].strip()

In [29]:
# Test the model
test_sentences = [
    "What are you doing this weekend?",
    "How's your current project going?",
    "How are you feeling today?",
    "What's your favorite childhood memory?",
    "What's your favorite movie and why?",
    "Where are you going?",
    "how is your health?",
    "How are you doing?",
    "Where are you going?",
    "What time is it?",
    "The food there is never any good.",
    "I could not walk any farther.",
    "I can't eat any more pizza.",

]

In [30]:
print("\nTesting the model:\n")
for sentence in test_sentences:
    translation = translate_to_Hindi_colloquial(sentence)
    print(f"English: {sentence}")
    print(f"Hindi Colloquial: {translation}")
    print("-" * 50)


Testing the model:

English: What are you doing this weekend?
Hindi Colloquial: ### Human: You are a Hindi colloquial language translator. Translate the following English text to Hindi colloquial language (spoken Tamil).
Here are some examples:
"What are you doing this weekend?" -> "aap is saptaahaant par kya kar rahe hain?"
"How's your current project going?" -> " aapaka vartamaan projekt kaisa chal raha hai?"
"How are you feeling today?" -> "aaj aap kaisa mahasoos kar rahe hain?"
"Can I have another cup of tea now please?" -> "naya tulika ek paani bhi milta hua aur meine dosto meeche nayega?"
"Do you like my cooking?" -> "jab mera pakkhoon thora chahi
--------------------------------------------------
English: How's your current project going?
Hindi Colloquial: ### Human: You are a Hindi colloquial language translator. Translate the following English text to Hindi colloquial language (spoken Tamil).
Here are some examples:
"What are you doing this weekend?" -> "aap is saptaahaant pa

In [31]:
from IPython.display import HTML, display

def interactive_translation(text_input):
    if text_input:
        translation = translate_to_Hindi_colloquial(text_input)
        display(HTML(f"""
        <div style='padding: 10px; border-radius: 5px;'>
            <b>English:</b> {text_input}<br>
            <b>Hindi Colloquial:</b> {translation}
        </div>
        """))

# Create interactive widget
from ipywidgets import interact, widgets
interact(interactive_translation,
         text_input=widgets.Text(description='English:', placeholder='Enter text to translate'))

interactive(children=(Text(value='', description='English:', placeholder='Enter text to translate'), Output())…

<function __main__.interactive_translation(text_input)>